# 最简单版本
使用 LR 模型进行预测，且无脑使用所有特征且不做转换

In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from scipy.stats import skew
from scipy.stats import pearsonr

%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

In [16]:
print(train.columns)

In [55]:
# 数据处理
# 1. 删除所有有 Nan 的列
# 2. 只选取类型为 number 的列
def process_data(df: pd.DataFrame):
    # axis=1: remove column
    df = df.dropna(axis=1, how='any')
    df = df.select_dtypes(include='number')
    return df
# 训练数据, 排除 SalePrice 列
train_data = train.loc[:, train.columns != 'SalePrice']
train_label = train.SalePrice
# 生成初识训练、测试数据集
train_processed = process_data(train_data)
test_processed = process_data(test)
# 选取两组数据集的列交接
columns = set(train_processed.columns)
columns = columns.intersection(set(test_processed.columns))
columns = list(columns)
# 生成最终用的训练、测试数据集
train_processed = train_processed[columns]
test_processed = test_processed[columns]

In [56]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_Y, test_Y = train_test_split(train_processed, train_label, test_size=0.3, random_state=10)
train_X.head()

In [58]:
from sklearn import linear_model as lm

# 训练 lr 模型
lr = lm.LogisticRegression()
lr.fit(train_X, train_Y)

In [59]:
from sklearn.metrics import accuracy_score
predict_Y = lr.predict(test_X)
accuracy_score(predict_Y, test_Y)

predict test dataset, and submit

In [60]:
text_predict_Y = lr.predict(test_processed)

In [61]:
print(len(text_predict_Y), len(test))

准备提交数据

In [65]:
!mkdir output

In [66]:
submission_data = pd.DataFrame({'Id': test.Id, 'SalePrice': text_predict_Y})
# print(my_submission)
submission_data.to_csv('output/submission.csv', index=False)

In [1]:
!ls output
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f output/submission.csv -m "lr"

score: 0.37272